# One-machines Federated Learning
Run all the cells, but first create a conda environment ```conda env create -f environment.yml``` (or manually and then run ```pip install -r requirements.txt```).


In [3]:
# Parameters
n_clients = 5
n_attackers = 1
attack_type = "MP_noise" # Choose between: 'DP_flip', 'DP_inverted_loss', 'MP_noise', 'MP_gradient'
n_rounds = 10
data_type = "2cluster" # Choose between: '2cluster', 'random', (i.e., non-IID and IID) -- synthetic can only go with random (already non-IID)
dataset = "mnist" # Choose between 'diabetes', 'breast', 'synthetic', 'mnist', 'cifar10'

## Data creation

In [4]:
import os
 
# Get the current working directory, remove last, and change the current working directory to the parent directory
current_path = os.getcwd()
parent_path = os.path.dirname(current_path) +'/data'
os.chdir(parent_path)
 
# # Create CIFAR-10 dataset
# print("\033[93m Create CIFAR-10\033[00m")
# script_path = os.path.join(parent_path, 'cifar_creation.py')
# !python $script_path
 
# Create MNIST dataset
print("\033[93m Create MNIST\033[00m")
script_path = os.path.join(parent_path, 'mnist_creation.py')
!python $script_path
 
# Split client datasets
parent_path = os.path.dirname(current_path)
os.chdir(parent_path)
print("\033[93m Split client datasets\033[00m")
script_path = os.path.join(parent_path, 'data/client_split.py')
!python $script_path --seed 3 --n_clients $n_clients
os.chdir(current_path)


 Create MNIST
/Users/dariofenoglio/miniforge3/envs/CF_FL/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/dariofenoglio/miniforge3/envs/CF_FL/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using device: mps
100%|█████████████████████████████████████████| 157/157 [00:02<00:00, 63.81it/s]
 Split client datasets


Data creation
Number of clients: 5
Random seed: 3
Diabetes dataset: (70692, 22)
Breast cancer dataset: (569, 32)
MNIST dataset: (10000, 1001)
CIFAR10 dataset: (10000, 1001)
Mi

## Training

In [5]:
import subprocess
import time

# Fixed parameters for this example
model = "net"
pers = "0"

# Start server process
server_command = [
    "python", "server_FBPs.py", 
    "--rounds", str(n_rounds), 
    "--data_type", data_type, 
    "--dataset", dataset, 
    "--model", model, 
    "--pers", pers, 
    "--n_clients", str(n_clients), 
    "--n_attackers", str(n_attackers), 
    "--attack_type", attack_type
]
server_process = subprocess.Popen(server_command)
time.sleep(2)

client_processes = []
for i in range(n_clients):
    print("Start Client", i+1)
    # Start client process
    client_command = [
        "python", "client.py", 
        "--id", str(i+1), 
        "--data_type", data_type, 
        "--model", model, 
        "--dataset", dataset
    ]
    client_processes.append(subprocess.Popen(client_command))
    
    
client_malicious_processes = []
for i in range(n_attackers):
    print("Start Attacker", i+1)
    # Start client process
    client_command = [
        "python", "malicious_client.py", 
        "--id", str(i+1), 
        "--data_type", data_type, 
        "--model", model, 
        "--dataset", dataset, 
        "--attack_type", attack_type
    ]
    client_malicious_processes.append(subprocess.Popen(client_command))
    

server_process.wait()
for p in client_processes:
    p.wait()
for p in client_malicious_processes:
    p.wait()



------- Federated Behavioural Planes (FBPs) -------

Used Size Server-Test Set: torch.Size([300, 1000])


INFO flwr 2024-11-11 13:44:39,770 | app.py:163 | Starting Flower server, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO flwr 2024-11-11 13:44:39,782 | app.py:176 | Flower ECE: gRPC server running (10 rounds), SSL is disabled
INFO flwr 2024-11-11 13:44:39,782 | server.py:89 | Initializing global parameters
INFO flwr 2024-11-11 13:44:39,782 | server.py:280 | Requesting initial parameters from one random client


Start Client 1
Start Client 2
Start Client 3
Start Client 4
Start Client 5
Start Attacker 1
MPS is available
MPS is available
MPS is available
MPS is available
MPS is available
MPS is available
Client 1 - Data type: 2cluster - Dataset: mnist - Model: net - Data shape: torch.Size([860, 1000])
Client 5 - Data type: 2cluster - Dataset: mnist - Model: net - Data shape: torch.Size([1321, 1000])
Client 2 - Data type: 2cluster - Dataset: mnist - Model: net - Data shape: torch.Size([1349, 1000])
Client 4 - Data type: 2cluster - Dataset: mnist - Model: net - Data shape: torch.Size([1444, 1000])
Client 3 - Data type: 2cluster - Dataset: mnist - Model: net - Data shape: torch.Size([1812, 1000])


INFO flwr 2024-11-11 13:44:42,237 | grpc.py:52 | Opened insecure gRPC connection (no certificates were passed)
INFO flwr 2024-11-11 13:44:42,237 | grpc.py:52 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2024-11-11 13:44:42,237 | connection.py:42 | ChannelConnectivity.IDLE
DEBUG flwr 2024-11-11 13:44:42,238 | connection.py:42 | ChannelConnectivity.IDLE
DEBUG flwr 2024-11-11 13:44:42,238 | connection.py:42 | ChannelConnectivity.CONNECTING
DEBUG flwr 2024-11-11 13:44:42,238 | connection.py:42 | ChannelConnectivity.READY
DEBUG flwr 2024-11-11 13:44:42,244 | connection.py:42 | ChannelConnectivity.READY
INFO flwr 2024-11-11 13:44:42,255 | grpc.py:52 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2024-11-11 13:44:42,256 | connection.py:42 | ChannelConnectivity.IDLE
DEBUG flwr 2024-11-11 13:44:42,256 | connection.py:42 | ChannelConnectivity.CONNECTING
INFO flwr 2024-11-11 13:44:42,259 | grpc.py:52 | Opened insecure gRPC connection (no c

Saving round 1 aggregated_parameters...


...

DEBUG flwr 2024-11-11 13:44:52,271 | server.py:177 | evaluate_round 9: strategy sampled 6 clients (out of 6)
DEBUG flwr 2024-11-11 13:44:52,404 | server.py:191 | evaluate_round 9 received 6 results and 0 failures
DEBUG flwr 2024-11-11 13:44:52,404 | server.py:226 | fit_round 10: strategy sampled 6 clients (out of 6)
DEBUG flwr 2024-11-11 13:44:52,580 | server.py:240 | fit_round 10 received 6 results and 0 failures


Saving round 10 aggregated_parameters...


DEBUG flwr 2024-11-11 13:44:53,082 | server.py:177 | evaluate_round 10: strategy sampled 6 clients (out of 6)
DEBUG flwr 2024-11-11 13:44:53,234 | server.py:191 | evaluate_round 10 received 6 results and 0 failures
INFO flwr 2024-11-11 13:44:53,234 | server.py:157 | FL finished in 10.952893834008137
INFO flwr 2024-11-11 13:44:53,236 | app.py:226 | app_fit: losses_distributed [(1, 0.7084064559316187), (2, 0.7094291135622043), (3, 0.6906703480732478), (4, 0.6653626553690919), (5, 0.6583917068089811), (6, 0.6797105121575179), (7, 0.6309587943018292), (8, 0.6259760669935709), (9, 0.6111586168797661), (10, 0.6088692207695174)]
INFO flwr 2024-11-11 13:44:53,236 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-11-11 13:44:53,236 | app.py:228 | app_fit: metrics_distributed {'accuracy': [(1, 0.457680261239425), (2, 0.4707419026639145), (3, 0.5083594517149771), (4, 0.6097178696776383), (5, 0.6342737733570386), (6, 0.5626959269136471), (7, 0.619122257551554), (8, 0.63166144885737

Training time: 0.22 minutes

Client 3 
Minimum Loss occurred at round 10 with a loss value of 0.4337715208530426 
Maximum Accuracy occurred at round 10 with an accuracy value of 0.9801324605941772 
Validity occurred at round 1 with a validity value of 0.9205297827720642


Client 1 
Minimum Loss occurred at round 9 with a loss value of 0.4429850876331329 
Maximum Accuracy occurred at round 9 with an accuracy value of 0.8930232524871826 
Validity occurred at round 2 with a validity value of 0.8232558369636536


Client 4 
Minimum Loss occurred at round 6 with a loss value of 0.6392081379890442 
Maximum Accuracy occurred at round 3 with an accuracy value of 0.6961326003074646 
Validity occurred at round 1 with a validity value of 0.9475138783454896


Client 2 
Minimum Loss occurred at round 2 with a loss value of 0.626721203327179 
Maximum Accuracy occurred at round 1 with an accuracy value of 0.8047337532043457 
Validity occurred at round 2 with a validity value of 0.8875739574432373


Cl

100%|██████████| 10/10 [00:00<00:00, 17.37it/s]
